<a href="https://colab.research.google.com/github/Andrian0s/ML4NLP1-2025-Tutorial-Notebooks/blob/main/exercises/ex4/ex4_ner_gliner_given_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

****ZERO shot NER and anonymization with GLiNER****

We will use a ***[fine-tuned version of GLiNER](https://huggingface.co/urchade/gliner_multi_pii-v1)***, specialized in personally identifiable information (PII).

⚡ GOAL: Create our own small dataset with various PII mentions, and anonymize it with GLiNER!

# Load the model

In [ ]:
from gliner import GLiNER

# NOTE: No need to load the model on GPU for our small dataset
model = GLiNER.from_pretrained("urchade/gliner_multi_pii-v1")


In [ ]:
# Code copy-pasted from the model card
text = """
Harilala Rasoanaivo, un homme d'affaires local d'Antananarivo, a enregistré une nouvelle société nommée "Rasoanaivo Enterprises" au Lot II M 92 Antohomadinika. Son numéro est le +261 32 22 345 67, et son adresse électronique est harilala.rasoanaivo@telma.mg. Il a fourni son numéro de sécu 501-02-1234 pour l'enregistrement.
"""

labels = ["work", "booking number", "personally identifiable information", "driver licence", "person", "book", "full address", "company", "actor", "character", "email", "passport number", "Social Security Number", "phone number"]
entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

Harilala Rasoanaivo => person
Rasoanaivo Enterprises => company
Lot II M 92 Antohomadinika => full address
+261 32 22 345 67 => phone number
harilala.rasoanaivo@telma.mg => email
501-02-1234 => Social Security Number


# Anonymize entities

In [ ]:
# TODO: Implement anonymization function
def anonymize_entities(text: str, entities: list[dict]) -> str:
    """Anonymize entities in text by replacing them with tags like <PERSON>, <IBAN> etc.

    Args:
        text (str): The original text
        entities (list): List of entity dictionaries with 'start', 'end', and 'label' keys

    Returns:
        str: Text with entities replaced by the corresponding tags.
    """
    pass

# Dataset

Time to create our own, diverse dataset. Make sure to not make it easy for the model (for example, do not mention the label in the sentence, but make sure it can be inferred from the context).

In [ ]:
# TODO: Fill these 2 lists with your own data. pii_texts should be of length (at least) 10.

pii_texts = []

labels = []

In [ ]:
# TODO: Create dataset
dataset: list[dict] = []

Time to see the results!

In [ ]:
for example in dataset:
    print("Original Text:\n", example["text"])
    print("Anonymized Text:\n", example["anonymized_text"])
    print("\n---\n")

# Report

📝❓Discuss the benefits of using GLiNER vs a more traditional NER solution.

📝❓Did you encounter false positives/negatives? Discuss.